## 使用Hyperdrive进行超参数调整
Hyperdrive是Azure机器学习提供的高级超参数调整功能。在Compute Cluster上，以并行方式进行高速学习，并搜索高精度的超参数组合。

**搜索算法**
- Gird Search
- Random Search
- Beysian Optimization    


※ 参考文件 : [使用Azure机器学习调整模型超参数](https://docs.microsoft.com/zh-cn/azure/machine-learning/how-to-tune-hyperparameters)

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### 获取数据集

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')

### 实验名称设定

In [ ]:
experiment = Experiment(workspace=ws, name='dummy-hyperdrive2')

### 训练脚本执行设置

In [ ]:

config = ScriptRunConfig(source_directory='./code/pytorch-hyperdrive',
                         script='train.py',
                         compute_target='compute1',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_mount(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])


### 环境调用和执行设置的输入

In [ ]:
env = Environment.get(ws, "pytorch-env")

In [ ]:
config.run_config.environment = env

### Hyperdrive中的参数设置

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

# 参数搜索范围设定
ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-6, -1),
        '--momentum': loguniform(-6, -1),
    }
)

In [ ]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

### Hyperdrive 実行設定

In [ ]:
hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='train_loss',  # 调整目标指标
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,  # or MAXIMIZE
                                     max_total_runs=20,  # 最大试验次数
                                     max_concurrent_runs=4)  # 最大平行

### 执行和结果确认

访问Jupyter Widget和Azure Machine Learning Studio的可视化功能以查看结果。

In [ ]:
run = experiment.submit(hyperdrive_config)

In [ ]:
# Jupyter Widgets
RunDetails(run).show()

In [ ]:
# 文字输出
run.wait_for_completion(show_output=True)